Code to find the average FA/MD/AD/RD per track, and its significance

Normalize by pseudo global means

In [ ]:
import csv
import numpy as np
import os
import random as rd
import re
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import mannwhitneyu

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator

In [ ]:
os.chdir('/data/Reback-DTI/')

In [ ]:
outcomes = ['pcpcgp_6mo', 'mortality_6mos']
measures = ['fa', 'rd', 'md', 'ad']
agerange = ['all_ages','ages0_1','ages1_5', 'ages0_5', 'ages0_2']

batch1_tracts = ['Cingulum_Parahippocampal_L','Cingulum_Parahippocampal_R','Inferior_Fronto_Occipital_Fasciculus_L','Inferior_Fronto_Occipital_Fasciculus_R','Inferior_Longitudinal_Fasciculus_L','Inferior_Longitudinal_Fasciculus_R','Corticospinal_Tract_L','Corticospinal_Tract_R','Corpus_Callosum_Body']
batch2_tracts = ['Cingulum_Parahippocampal_L','Cingulum_Parahippocampal_R','Inferior_Fronto_Occipital_Fasciculus_L','Inferior_Fronto_Occipital_Fasciculus_R','Inferior_Longitudinal_Fasciculus_L','Inferior_Longitudinal_Fasciculus_R','Corticospinal_Tract_L','Corticospinal_Tract_R','Corpus_Callosum_Body','Cingulum_Frontal_Parahippocampal_L','Cingulum_Frontal_Parahippocampal_R','Cingulum_Frontal_Parietal_L','Cingulum_Frontal_Parietal_R','Cingulum_Parahippocampal_Parietal_L','Cingulum_Parahippocampal_Parietal_R','Cingulum_Rarolfactory_L','Cingulum_Rarolfactory_R','Corpus_Callosum_Forceps_Minor','Corpus_Callosum_Tapetum','Corpus_Callosum_Forceps_Major']

In [ ]:
Eddy_corrected = '/data/Reback-DTI/resources/stats/Eddy_corrected/'
Noncorrected = '/data/Reback-DTI/resources/stats/Noncorrected'
Batch1 = os.path.join(Eddy_corrected, 'Batch1')
Batch2 = os.path.join(Eddy_corrected, 'Batch2')

#***********SELECT YOUR CHOICE****************#
selected_agerange = agerange[0]
Batch_dir  = Batch1
#***********SELECT YOUR CHOICE****************#

if Batch_dir == Batch1:
    tracts = batch1_tracts
elif Batch_dir == Batch2:
    tracts = batch2_tracts

datasrc_dir = os.path.join(Batch_dir, 'output_dir', 'strats', selected_agerange)
datasrc_file = os.path.join(datasrc_dir, '{}.csv'.format(selected_agerange))

outputBaseDir = os.path.join(datasrc_dir, 'normalByMean')
if not os.path.exists(outputBaseDir):
    os.makedirs(outputBaseDir)

In [ ]:
def getData(src):
    pdata = pd.read_csv(src)
    pdata['id'] = pdata['StudyID'].map(str) + '_' + pdata['MR_num'].map(str) + '_' + pdata['Scan_num'].map(str)
    #place_holder for future, won't currently delete anything probably
    pdata = pdata.drop_duplicates(subset='id')  
    pdata.head()
    return pdata

def makeEmptyResultsDf():
    column_names = ['tract', 'outcome', 'measure', 'pvalue']
    results_df = pd.DataFrame(columns = column_names)
    return results_df

#note, this is different from the one in statistics.ipynb
def getTractMeasure(pdata, outcome=outcomes[0],tract='', measure='fa'): #default is all tracts FA
    tract_measure= re.compile(r''+tract+'.{}$'.format(measure))
    filtered = list(filter(tract_measure.search, pdata.columns))
    #print(filtered)

    ### USE MELT TO CONVERT FROM WIDE TO LONG FORM
    df_meas = pd.melt(pdata, id_vars=['id', outcome, 'Sex'], value_vars=filtered, var_name='Tract', value_name='{}'.format(measure)) 
    df_meas['Tract'] = df_meas['Tract'].str.strip('.{}'.format(measure))
    df_meas = df_meas.dropna()
    df_meas.head()
    return df_meas 

def getNormByMean(pdata, outcome=outcomes[0],tract='', measure='fa'): #default is all tracts FA
    tract_measure= re.compile(r''+tract+'.{}$'.format(measure))
    filtered = list(filter(tract_measure.search, pdata.columns))

    reqFields = ['id', outcome, 'Sex']
    reqFields.extend(filtered)
    df_nbm = pdata[reqFields]
    
    numtracts = len(tracts)
    lbound = 0-numtracts
    ubound = 0
    df_nbm['globalAvg'] = df_nbm.iloc[:, lbound:].sum(axis=1).divide(numtracts)
    df_nbm_vals = df_nbm.iloc[:, lbound-1:ubound-1].divide(df_nbm.globalAvg, axis=0)
    df_nbm[df_nbm_vals.columns] = df_nbm_vals
    
    return df_nbm 

In [ ]:
datapd     = getData(datasrc_file)
#test
datapd_nbm = getNormByMean(datapd, outcomes[0], measure=measures[0])
datapd_nbm

In [ ]:
results_df = makeEmptyResultsDf()
df_fa  = None #for future use
df_rd  = None #for future use
df_md  = None #for future use
df_ad  = None #for future use
df_age = None #for future use
df_sex = None #for future use
#get norm for every measure
for meas in measures:
    measure    = meas
    datapd     = getData(datasrc_file)
    datapd_nbm = getNormByMean(datapd, outcomes[0], measure=measure)
    
    ####for future use###
    if measure=='fa':
        df_fa = datapd_nbm
    if measure=='rd':
        df_rd = datapd_nbm
    if measure=='md':
        df_md = datapd_nbm
    if measure=='ad':
        df_ad = datapd_nbm
    df_age = datapd[['id', outcomes[0], 'Age_y']]
    df_age['Tract'] = 'All'
    df_sex = datapd[['id', outcomes[0], 'Sex']]
    df_sex['Tract'] = 'All'
    #####################
    
    outputFilename = os.path.join(outputBaseDir, 'nbm_{}.csv'.format(measure))
    datapd_nbm.to_csv(outputFilename, sep=',', index=False)

    outcome0 = datapd_nbm.loc[(datapd_nbm[outcomes[0]] == 0)]# & (datapd_nbm['Sex'] == 0) ]
    outcome1 = datapd_nbm.loc[(datapd_nbm[outcomes[0]] == 1)]# & (datapd_nbm['Sex'] == 0) ]

    for tract in tracts:
        U1, pval = stats.mannwhitneyu(outcome0['{}.{}'.format(tract,measure)], outcome1['{}.{}'.format(tract,measure)])
        
        row = {'tract': tract, 'outcome': outcomes[0], 'measure': measure, 'pvalue': pval}
        results_df = results_df.append(row, ignore_index=True)
    
        
results_wide = results_df.pivot(index=['tract'], columns='measure', values='pvalue')
results_wide.reset_index()

In [ ]:
# results_df = makeEmptyResultsDf()
# df_fa  = None #for future use
# df_rd  = None #for future use
# df_md  = None #for future use
# df_ad  = None #for future use
# df_age = None #for future use
# df_sex = None #for future use
# for meas in measures:
#     measure    = meas
#     datapd     = getData(datasrc_file)
#     datapd_nbm = getNormByMean(datapd, outcomes[0], measure=measure)
    
#     ####for future use###
#     if measure=='fa':
#         df_fa = datapd_nbm
#     if measure=='rd':
#         df_rd = datapd_nbm
#     if measure=='md':
#         df_md = datapd_nbm
#     if measure=='ad':
#         df_ad = datapd_nbm
#     df_age = datapd[['id', outcomes[0], 'Age_y']]
#     df_age['Tract'] = 'All'
#     df_sex = datapd[['id', outcomes[0], 'Sex']]
#     df_sex['Tract'] = 'All'
#     #####################
    
#     outputFilename = os.path.join(outputBaseDir, 'nbm_{}.csv'.format(measure))
#     datapd_nbm.to_csv(outputFilename, sep=',', index=False)

#     outcome0 = datapd_nbm.loc[(datapd_nbm[outcomes[0]] == 0)]# & (datapd_nbm['Sex'] == 0) ]
#     outcome1 = datapd_nbm.loc[(datapd_nbm[outcomes[0]] == 1)]# & (datapd_nbm['Sex'] == 0) ]

#     for tract in tracts:
#         result = stats.ttest_ind(outcome0['{}.{}'.format(tract,measure)], outcome1['{}.{}'.format(tract,measure)])
        
#         row = {'tract': tract, 'outcome': outcomes[0], 'measure': measure, 'pvalue': result.pvalue}
#         results_df = results_df.append(row, ignore_index=True)
        
# results_wide = results_df.pivot(index=['tract'], columns='measure', values='pvalue')
# results_wide.reset_index()

In [ ]:
pvalue_output = os.path.join(outputBaseDir, 'pvals_{}.txt'.format(selected_agerange)) 
results_wide.to_csv(pvalue_output)

Violin Plots

In [ ]:
def plotViolins(df_fa, df_rd, df_md, df_ad, saveFile=None):
    plt.style.use('ggplot')
    fig = plt.figure(figsize=(16, 16))

    plt.subplot(411) 
    ax1 = sns.violinplot(x='Tract', y='fa', data=df_fa, hue=outcomes[0], split=True, scale="count", inner="stick", scale_hue=False, bw=.2)
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)
    ax1.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    ax1.set(xticklabels=[])
    ax1.set(xlabel=None)

    plt.subplot(412) 
    ax1 = sns.violinplot(x='Tract', y='rd', data=df_rd, hue=outcomes[0], split=True)
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)
    ax1.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    ax1.set(xticklabels=[])
    ax1.set(xlabel=None)

    plt.subplot(413) 
    ax1 = sns.violinplot(x='Tract', y='md', data=df_md, hue=outcomes[0], split=True)
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)
    ax1.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    ax1.set(xticklabels=[])
    ax1.set(xlabel=None)

    plt.subplot(414) 
    ax1 = sns.violinplot(x='Tract', y='ad', data=df_ad, hue=outcomes[0], split=True)
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)
    ax1.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    plt.setp(ax1.get_xticklabels(), rotation=30, horizontalalignment='right')
    
    if saveFile != None:
        plt.savefig(saveFile)

def plotCovariates(df_age, df_sex, saveFile=None):
    plt.style.use('ggplot')
    fig = plt.figure(figsize=(9, 16))

    plt.subplot(311) 
    ax1 = sns.boxplot(x=outcomes[0], y="Age_y", data=df_age)
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)
    sns.stripplot(x = outcomes[0], y = "Age_y", color = 'black', data = df_age)

    plt.subplot(312) 
    ax1 = sns.barplot(x='Tract', y='Sex', data=df_sex, hue=outcomes[0])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)

    plt.subplot(313) 
    ax1 = sns.countplot(x=outcomes[0], data=df_age)
    
    if saveFile != None:
        plt.savefig(saveFile)

In [ ]:
def plotGroupBoxplots(df_fa, df_rd, df_md, df_ad, saveFile=None):
    
    plt.style.use('ggplot')
    fig = plt.figure(figsize=(16, 16))
    
    plt.subplot(411)
    ax = sns.boxplot(x="Tract", y="fa",
            hue=outcomes[0], palette='Pastel1',
            data=df_fa)
    ax = sns.despine(offset=10, trim=True)
    ax = sns.swarmplot(x="Tract", y="fa", data=df_fa, hue=outcomes[0], dodge=True)
    ax.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)

    
    plt.subplot(412)
    ax = sns.boxplot(x="Tract", y="rd",
            hue=outcomes[0], palette='Pastel1',
            data=df_rd)
    ax = sns.despine(offset=10, trim=True)
    ax = sns.swarmplot(x="Tract", y="rd", data=df_rd, hue=outcomes[0], dodge=True)
    ax.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)

    
    plt.subplot(413)
    ax = sns.boxplot(x="Tract", y="md",
            hue=outcomes[0], palette='Pastel1',
            data=df_md)
    ax = sns.despine(offset=10, trim=True)
    ax = sns.swarmplot(x="Tract", y="md", data=df_md, hue=outcomes[0], dodge=True)
    ax.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)
    
    plt.subplot(414)
    ax = sns.boxplot(x="Tract", y="ad",
            hue=outcomes[0], palette='Pastel1',
            data=df_ad)
    ax = sns.despine(offset=10, trim=True)
    ax = sns.swarmplot(x="Tract", y="ad", data=df_ad, hue=outcomes[0], dodge=True)
    ax.axhline(1.00, alpha=0.5, c='#000000', ls='-')
    #ax.set(xticklabels=[])
    ax.set(xlabel=None)
    
    
    plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right') 

    
    if saveFile != None:
        plt.savefig(saveFile)

In [ ]:
df_fa_l = getTractMeasure(df_fa, measure=measures[0])
df_rd_l = getTractMeasure(df_rd, measure=measures[1])
df_md_l = getTractMeasure(df_md, measure=measures[2])
df_ad_l = getTractMeasure(df_ad, measure=measures[3])

In [ ]:
violin_output    = None
covariate_output = None
violin_output    = os.path.join(outputBaseDir, 'normByAvgBoxplotsFA_{}.png'.format(selected_agerange))
#covariate_output = os.path.join(outputBaseDir, 'normByAvgViolin_covariates_{}.png'.format(selected_agerange))

#plotViolins(df_fa_l, df_rd_l, df_md_l, df_ad_l, violin_output)
#plotCovariates(df_age, df_sex, covariate_output)
plotGroupBoxplots(df_fa_l, df_rd_l, df_md_l, df_ad_l, violin_output)